In [6]:
import pandas as pd
import numpy as np
import os
import glob

In [7]:
input = 'INPUT//'
input_cust = 'INPUT//CUST'
input_group = 'INPUT//GROUP'
output = 'OUTPUT//'
mapping = 'MAPPING//'
# os.mkdir(input)
# os.mkdir(output)
# os.mkdir(mapping)
# os.mkdir(input_cust)
# os.mkdir(input_group)

### backup

In [8]:
# mapping_list = pd.ExcelFile(glob.glob(os.path.join(mapping,'*xlsx'))[-1])
# # สร้างพจนานุกรมเพื่อเก็บ DataFrame
# dfs = {sheet_name: pd.read_excel(mapping_list, sheet_name=sheet_name) for sheet_name in mapping_list.sheet_names}
# # ดู sheets name ทั้งหมด
# # dfs.keys()

# mapping_list = pd.ExcelFile(glob.glob(os.path.join(mapping,'*xlsx'))[-1])
# dfs = {}
# for sheet_name in mapping_list.sheet_names:
#     data = pd.read_excel(mapping_list, sheet_name=sheet_name)
#     dfs[sheet_name] = data


# mapping_list = pd.ExcelFile(glob.glob(os.path.join(mapping,'*xlsx'))[-1])
# for sheet_name in mapping_list.sheet_names:    
#     exec(f"{sheet_name} = pd.read_excel(mapping_list, sheet_name=sheet_name)")

### function

In [9]:
def import_cust():
    cust = pd.read_excel(glob.glob(os.path.join(input_cust,'*xlsx'))[-1])
    return cust

def import_group():
    group = pd.read_excel(glob.glob(os.path.join(input_group,'*xlsx'))[-1],dtype={'CUST_GROUP': str,'CUST_CODE': str})
    return group

def import_mapping():
    mapping_list = pd.ExcelFile(glob.glob(os.path.join(mapping, '*xlsx'))[-1])
    mapping_dataframes = {}
    for sheet_name in mapping_list.sheet_names:
        ### เก็บข้อมูล แบบ dic เรียกใช้โดย ชื่อที่เรียกมาเก็บ[sheet_name]
        mapping_dataframes[sheet_name] = pd.read_excel(mapping_list, sheet_name=sheet_name,dtype={'CUST_GROUP': str,'รหัสธุรกิจ':str})
        # mapping_dataframes[sheet_name] = mapping_dataframes[sheet_name].drop_duplicates(subset=['CUST_GROUP','stage'])
    return mapping_dataframes

def convert_stage(df):
    condition = [df.str[-1] == '1'
                    ,df.str[-1] == '2'
                    ,df.str[-1] == '3']
    return np.select(condition,[ 1, 2 , 3])

def convert_stage2(df):
    condition = [df == 1
                    ,df == 2
                    ,df == 3]
    return np.select(condition,[ 'stage1', 'stage2' , 'stage3'])

def stage_final(df):
    condition = [~df['stage_NPL'].isna()
                ,(df['stage_NPL'].isna())&(~df['stage_ECL'].isna())
                ,(df['stage_NPL'].isna())&(df['stage_ECL'].isna())&(~df['stage_LG'].isna())
                ,(df['stage_NPL'].isna())&(df['stage_ECL'].isna())&(df['stage_LG'].isna())&(~df['stage_watchlist'].isna())
                ]
    choices = [df['stage_NPL'],df['stage_ECL'],df['stage_LG'],df['stage_watchlist']]
    return np.select(condition,choices,default=1)
     

### stage_flag

In [10]:
cust = import_cust()
group = import_group()
mapping_file = import_mapping()
mapping_file['NPL']['stage'] = convert_stage(mapping_file['NPL']['stage'])
mapping_file['ECL']['stage'] = convert_stage(mapping_file['ECL']['stage'])
mapping_file['LG']['stage'] = convert_stage(mapping_file['LG']['stage'])
mapping_file['watchlist']['stage'] = convert_stage(mapping_file['watchlist']['stage'])
# mapping_file['ECL_Interbank']['Stage'] = clean_stage(mapping_file['ECL_Interbank']['Stage'])
mapping_file['ECL_Interbank']['Bank'] = mapping_file['ECL_Interbank']['Bank'].str.strip()
group.rename({'CUST_GROUP':'CUST_GROUP_M','CUST_CODE':'CUST_GROUP'},axis= 'columns',inplace = True)
mapping_file['NPL'] = mapping_file['NPL'].drop_duplicates(subset=['CUST_GROUP','stage'])
mapping_file['ECL'] = mapping_file['ECL'].drop_duplicates(subset=['CUST_GROUP','stage'])
mapping_file['LG'] = mapping_file['LG'].drop_duplicates(subset=['CUST_GROUP','stage'])
group['stage_NPL'] = group.merge(mapping_file['NPL'].iloc[:,[0,3]],on='CUST_GROUP',how = "left")['stage']
group['stage_ECL'] = group.merge(mapping_file['ECL'],on='CUST_GROUP',how='left')['stage']
group['stage_LG'] = group.merge(mapping_file['LG'],on='CUST_GROUP',how='left')['stage']
group['stage_watchlist'] = group.merge(mapping_file['watchlist'],on='CUST_GROUP',how='left')['stage']
group['stage'] = stage_final(group)
group = group.groupby(['CUST_GROUP_M'])[['stage']].max()
group.reset_index(inplace=True)
group.rename(columns={'CUST_GROUP_M': 'CUST_GROUP'},inplace=True)
# group['stage'] = convert_stage2(group['stage'])
group.to_excel(os.path.join(output,'group_stage.xlsx'))
## stage
cust['stage_group'] = cust.merge(group,on=['CUST_GROUP'],how='left')['stage']
cust['stage_NPL'] = cust.merge(mapping_file['NPL'].iloc[:,[0,3]],on='CUST_GROUP',how = "left")['stage']
cust['stage_ECL'] = cust.merge(mapping_file['ECL'],on='CUST_GROUP',how='left')['stage']
cust['stage_LG'] = cust.merge(mapping_file['LG'],on='CUST_GROUP',how='left')['stage']
cust['stage_watchlist'] = cust.merge(mapping_file['watchlist'],on='CUST_GROUP',how='left')['stage']
cust['stage_cust'] = stage_final(cust)
cust['stage_final'] = np.where(~(cust['stage_group'].isna())&cust['stage_cust']<cust['stage_group'],cust['stage_group'],cust['stage_cust'])
cust['stage_final'] = convert_stage2(cust['stage_final'])
## Grade
cust['Composite Raing Raing_adj'] = np.where(~(cust['Composite Raing'].isin(['A','B1','B2','B3', 'B4','C1', 'C2', 'C3']))|(cust['Year']<2020)|(cust['Composite Raing'].isna()),'N/A',cust['Composite Raing'])
cust.groupby(['Composite Raing Raing_adj']).size()
## isic code

c:\Anaconda 3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Composite Raing Raing_adj
A       135
B1      612
B2     1743
B3     2625
B4     2806
C1     1408
C2      107
C3      285
N/A    1675
dtype: int64